# Understanding basic siibra concepts

In [ ]:
import siibra
from nilearn import plotting

## Spaces, parcellations and regions

At its core, `siibra` defines different atlases, maps, and reference spaces as semantic concepts. Predefined objects of these can be accessed via registries that provide easy access via autocompletion or keyword matching.

In [ ]:
print("Predefined atlases",siibra.atlases)

# fetch an object using explicit autocompletion
atlas = siibra.atlases.MULTILEVEL_HUMAN_ATLAS 

# fetch via keyword matching - same result
atlas = siibra.atlases['human']
print(atlas)

In [ ]:
print("Predefined spaces",siibra.spaces)
bigbrain = siibra.spaces['bigbrain']
print(bigbrain)

An atlas object provides easy access to a consistent subset of available parcellations, reference spaces and brain regions. It will use reasonable defaults, e.g. return the most recent version of a parcellation if multiple are available.

In [ ]:
julichbrain = atlas.get_parcellation('julich')
print(type(julichbrain))
print(julichbrain)

The different objects are typically linked to curated datasets in the [EBRAINS knowledge graph](https://kg.ebrainseu) (KG), using dataset identifiers. They contain various metadata.

In [ ]:
print(julichbrain.name)
print(julichbrain.id)
print()
print(julichbrain.description)
print()
for p in julichbrain.publications:
    print(p['citation'])

`siibra` also represents brain regions as semantic objects using the specific class `Region`.

In [ ]:
v1 = atlas.get_region("v1")
print(type(v1))
print(v1)

Regions are part of a hierarchical tree. They may thus have parent and child regions:

In [ ]:
print(v1.parent.name)
print(v1.children)

In fact, each region represents its own subtree. This can be easily visualized by printing its full representation:

In [ ]:
v1

In [ ]:
v1.parent

## Maps

Spaces and parcellations are just semantic objects - they define a concept and provide desriptions. We are typically interested to combine them, i.e. to work with a concrete parcellation map in a particular reference space. Since EBRAINS atlases provide maps of some parcellations in multiple spaces, maps and parcellations are different concepts - maps are spatial objects, while parcellations are pure semantic objects.

In [ ]:
jubrain_mpm = atlas.get_map(siibra.spaces['mni152'])
print(jubrain_mpm)
print(f"This parcellation map defines {len(jubrain_mpm)} maps.")

`siibra` typically performs lazy loading of data, so that data is only retrieved when requested. As a consequence, in order to access the actual parcellation image, we call `fetch`, or `fetchall` if the parcellation defines multiple maps. The resulting objects are a `nibabel` standard type, and can thus be directly used in many common software libraries. Here we display the maximum probability map of the Julich-Brain probabilistic maps, which comes in separate images for the left and right hemisphere.

In [ ]:
from nilearn import plotting
for img in  jubrain_mpm.fetchall():
    plotting.plot_stat_map(img)

In [ ]:
# Try to display maps of long fiber bundles in mni152 space!
bundlemap = atlas.get_map(space='mni152',parcellation='long bundles').fetch()
plotting.plot_stat_map(bundlemap)

Probabilistic atlases provide a richer continuous representation of each areal map, whicih can be accessed by explicitly selecting the continuous map type `siibra.MapType.CONTINUOUS` when requesting the map.

In [ ]:
pmaps = atlas.get_map(space='mni152',
                      parcellation='julich 2.9',
                      maptype=siibra.MapType.CONTINUOUS)
print(pmaps)
print(f"This map defines {len(pmaps)} different maps.")

Of course, maps and label indices of maps do represent brain regions. `siibra`'s map objects keep track of label and map indices appropriately, so we can always access the corresponding region object. This is crucial to avoid confusion between indices and regions, a common error source in data experiments that employ brain atlases.

As an example, assume we want to display the probabiliy map of v1 in the left hemisphere. Which one to fetch?

In [ ]:
# Which probability map encodes V1 in the left hemisphere?
index = pmaps.decode_region('v1 left')
plotting.plot_stat_map(pmaps.fetch(mapindex=120))

Vice versa, we can easily find the region of a given map index.

In [ ]:
region = pmaps.decode_label(120)
print(region)

The same works accordingly for the maximum probabily map, where the region is encoded by both the map index (here: the hemisphere) and its color, i.e. label index: 

In [ ]:
cytomap.decode_label(mapindex=0,labelindex=10)

## Reference templates and high-resolution data

Besides brain maps, each reference space also has a data representation - typically an image - which represents the space. Interesting is particularly the BigBrain, which is a microscopic model and too large to download. For such cases, `siibra` allows to fetch high-resolution data at reduced resolutions, or fetching only volumes of interest.

In [ ]:
mni152 = atlas.get_template('mni152')
plotting.plot_img(mni152.fetch(),cmap='gray')

In [ ]:
bigbrain_space = atlas.get_space('bigbrain')
bigbrain = bigbrain_space.get_template()
plotting.view_img(bigbrain.fetch(),bg_img=None,cmap='gray')

In [ ]:
minpt = (-3.979, -61.256, 3.906)
maxpt = (5.863, -55.356, -2.487)
voi = bigbrain_space.get_bounding_box(minpt,maxpt)
chunk = bigbrain.fetch(resolution_mm=-1, voi=voi) # -1 means: highest resolution available
plotting.view_img(chunk,bg_img=None,cmap='gray')

We can reuse this region of interest, to fetch maps of cortical layers and display them with the BigBrain data.

In [ ]:
layers = atlas.get_map("bigbrain","layers").fetch(resolution_mm=-1,voi=voi)
plotting.view_img(layers,bg_img=chunk,opacity=.1,symmetric_cmap=False)